In [33]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import math
import time
import datetime as datetime
from tqdm import tqdm

Make sure that if you change any of the code segments related to a type of backtest to recompile the code segment changed before running the back test

In [34]:
uso_df = pd.read_csv('USO_date_cleaned.csv')
datelist = pd.read_csv('datelist.csv')
dates = []
for i,val in datelist.iterrows():
    dates.append(val[1])

Will calculate stats for the results of a benchmark. The input list is a 2 column dataframe of dates and the date's corresponding returns

In [35]:
def calculateStats(returns):
    # Average daily arithmetic returns
    avg_arth_daily_returns = returns['Returns'].mean()

    # Average daily geometric returns
    avg_geo_daily_returns = (stats.gmean(returns['Returns']))# Why doeos the final do ** 52 -1

    # Average annual arithmetic returns
    avg_arth_yearly_returns = (avg_arth_daily_returns - 1) * len(returns.index)/10

    # Average annual geometric returns
    
    # Standard Deviation
    std_daily_returns = returns.std()
    # In next line, 10 comes from having 10 years in data set, finding average number of days in year
    std_yearly_returns = returns.std()*math.sqrt(len(returns.index)/10)

    sharpe = (avg_arth_yearly_returns - .02)/std_yearly_returns

    print("Backtest Statistics")
    print("Average Arithmetic Daily Returns: {0:.3%}".format(float(avg_arth_daily_returns - 1)))
    print("Average Geometric Daily Returns: {0:.3%}".format(float(avg_geo_daily_returns - 1)))
    print("Average Arithmetic Yearly Returns: {0:.3%}".format(float(avg_arth_yearly_returns)))
    print("Standard Deviation of Daily Returns: {0:.3f}".format(float(std_daily_returns)))
    print("Standard Deviation of Yearly Returns: {0:.3f}".format(float(std_yearly_returns)))
    print("Sharpe ratio: {0:.3f}".format(float(sharpe)))



Parts of backtester for current day signal

In [36]:
def oneDay(open_close, date, cash):
    day = open_close[open_close['Date'] == date]
    prev_cash = cash
    rowiter = day.iterrows()
    for (i, row1), (j, row2) in zip(rowiter, rowiter):
        etfs_bought = 0
        if row1['Open'] < row1['Close']: # Buy
            # etfs_bought = cash/row1['Close']
            # cash -= etfs_bought * row1['Close']
            # etfs_sold_price = etfs_bought * row2['Close']
            # cash += etfs_sold_price

            etfs_sold = cash/row1['Close']
            cash += etfs_sold * row1['Close']
            etfs_bought_price = etfs_sold * row2['Close']
            cash -= etfs_bought_price
        else: # SELL
            # etfs_sold = cash/row1['Close']
            # cash += etfs_sold * row1['Close']
            # etfs_bought_price = etfs_sold * row2['Close']
            # cash -= etfs_bought_price

            etfs_bought = cash/row1['Close']
            cash -= etfs_bought * row1['Close']
            etfs_sold_price = etfs_bought * row2['Close']
            cash += etfs_sold_price
    returns = cash/prev_cash        
    return [cash, date, returns]

Test for oneDay for any chosen day

In [37]:
# Set start and end time intervals, initial day for one time test
start_interval = "09:30 - 10:00"
end_interval = "15:30 - 16:00"
day = "2012-01-03"
# Setting up dataframe with desired intervals
open = uso_df[uso_df['Time Interval'] == start_interval]
close = uso_df[uso_df['Time Interval'] == end_interval]
open_close = pd.concat([open, close])
open_close = open_close.sort_values(by='Date')
# Call oneDay
day = oneDay(open_close, day, 100)
print(day)

[99.46808510638297, '2012-01-03', 0.9946808510638298]


Method moreDays performs oneDay for all days in the dataframe

In [38]:
def moreDays(dates, open_close, start_interval, end_interval, cash):
    starting_cash = cash
    returns = pd.DataFrame(columns = ['Date', 'Returns'])
    open = uso_df[uso_df['Time Interval'] == start_interval]
    close = uso_df[uso_df['Time Interval'] == end_interval]
    open_close = pd.concat([open, close])
    open_close = open_close.sort_values(by='Date')
    for day in tqdm(dates):
        prev_cash = cash
        output = oneDay(open_close, day, cash)
        cash += output[0]
        cash -= prev_cash
        temp = pd.DataFrame([[output[1],output[2]]], columns = ['Date','Returns'])
        returns = returns.append(temp, ignore_index=True)
    print("Starting Cash: {0:.2f}".format(float(starting_cash)))
    print("Ending Cash: {0:.2f}".format(float(cash)))
    calculateStats(returns)
    print(cash)

Set up input for current day signal backtest

In [39]:
# Set start and end time intervals, initial day for one time test
start_interval = "09:30 - 10:00"
end_interval = "15:30 - 16:00"
cash = 1000
# Setting up dataframe with desired intervals
final = moreDays(dates, uso_df, start_interval, end_interval, cash)
print(final)

100%|██████████| 2552/2552 [00:03<00:00, 687.11it/s]

Starting Cash: 1000.00
Ending Cash: 4741.95
Backtest Statistics
Average Arithmetic Daily Returns: 0.072%
Average Geometric Daily Returns: 0.061%
Average Arithmetic Yearly Returns: 18.354%
Standard Deviation of Daily Returns: 0.015
Standard Deviation of Yearly Returns: 0.237
Sharpe ratio: 0.690
4741.9497348657005
None


Parts of the benchmark backtester

In [40]:
def oneDayBenchmark(open_close, date, cash):
    day = open_close[open_close['Date'] == date]
    
    rowiter = day.iterrows()
    for (i, row1) in rowiter:
        etfs_bought = cash/row1['Open']
        cash -= etfs_bought * row1['Open']
        etfs_sold_price = etfs_bought * row1['Close']
        cash += etfs_sold_price

    return cash

Test the oneDayBenchmark on any chosen day

In [41]:
interval = "15:30 - 16:00"
day = "2012-01-03"
# Setting up dataframe with desired intervals
open_close = uso_df[uso_df['Time Interval'] == interval]
open_close = open_close.sort_values(by = 'Date')
final = oneDayBenchmark(open_close, day, 100)
print(final)

99.97481108312341


Method moreDays to run oneDayBenchmark on every day

In [42]:
def moreDaysBenchmark(dates, uso_df, interval, cash):
    open_close = uso_df[uso_df['Time Interval'] == interval]
    open_close = open_close.sort_values(by = 'Date')
    for day in tqdm(dates):
        prev_cash = cash
        cash += oneDayBenchmark(open_close, day, cash)
        cash -= prev_cash
    return cash

Set up input for benchmark backtest

In [43]:
# Set up interval here
interval = "15:30 - 16:00"

# Setting up dataframe with desired intervals
final = moreDaysBenchmark(dates, uso_df, interval, cash)
print(final)

100%|██████████| 2552/2552 [00:01<00:00, 2014.32it/s]

1272.67320730087


Parts of backtester for today morning close/previous close signal. Our signal is the log(current day 10am close/previous day 4pm close), where if this value is greater than or equal to 0, you go long at 3:30pm today and sell at 4:00pm, and otherwise, you go short. 

In [62]:
# oneDay2 accepts a filtered dataframe open_close containing only rows with intervals equal to start/end intervals,
# i1 and i2 are indices for starting the backtest (do 1, 4 to perform entire backtest), and the amount of cash to 
# be used in the trade. Does not accept date cuz i1, i2, are used to access rows. Make sure i1 is odd or this fails
def oneDay2(open_close, i1, i2, cash):
    day = open_close.iloc[i1:i2]
    prev_cash = cash
    rowiter = day.iterrows()
    for (i, row1), (j, row2), (k, row3) in zip(rowiter, rowiter, rowiter):
        etfs_bought = 0

        # Create Signal
        p1 = row2['Close']
        p0 = row1['Close']
        r1 = math.log((p1/p0), 10)

        if r1 >= 0:
            # Long
            etfs_bought = int(cash/row3['Open'])
            cash -= etfs_bought * row3['Open']
            etfs_sold_price = etfs_bought * row3['Close']
            cash += etfs_sold_price
        else:
            # Short
            etfs_sold = int(cash/row3['Open'])
            cash += etfs_sold * row3['Open']
            etfs_bought_price = etfs_sold * row3['Close']
            cash -= etfs_bought_price
    
    percentReturns = (cash)/prev_cash
    return [cash, row3['Date'], percentReturns]

Test oneDay2 for any chosen day (chosen day will be used for previous day close, day after chosen day used for 10 am close and 3:30pm-4:00pm trade). Day chosen is picked by changing i1-i4 value

In [51]:
# Set desired Intervals
early_interval = "09:30 - 10:00" # On current day
late_interval = "15:30 - 16:00" # On previous day

# Set up dataframe with desired inputs
open = uso_df[uso_df['Time Interval'] == early_interval]
close = uso_df[uso_df['Time Interval'] == late_interval]
open_close = pd.concat([open, close])
open_close = open_close.sort_index()
open_close = open_close.sort_values(by='Date')
open_close = open_close.drop(['Unnamed: 0', 'index'], axis = 1)
open_close = open_close.reset_index(drop = True)

# Run backtest for oneDay2
i1 = 1
i2 = 4
day = oneDay2(open_close, i1,i2, 1000)
day

,Time Interval,Close,Net Chg,Open,High,Low,Tick Count,Volume,Date,Weekday
1,15:30 - 16:00,317.52,-0.08,317.60,318.00,317.44,3879.0,258519.0,2012-01-03,1
2,09:30 - 10:00,318.24,0.72,316.00,319.68,315.52,13579.0,668658.0,2012-01-04,2
3,15:30 - 16:00,318.16,-0.08,318.16,318.32,318.00,1719.0,61791.0,2012-01-04,2


moreDays2 will perform oneDay2 for all days 

In [46]:
def moreDays2(uso_df, dates, i1, i2, start_interval, end_interval, cash):
    returns = pd.DataFrame(columns=['Date','Returns'])
    starting_cash = cash
    open = uso_df[uso_df['Time Interval'] == start_interval]
    close = uso_df[uso_df['Time Interval'] == end_interval]
    open_close = pd.concat([open, close])
    open_close = open_close.sort_index()
    open_close = open_close.sort_values(by='Date')
    open_close = open_close.drop(['Unnamed: 0', 'index'], axis = 1)
    open_close = open_close.reset_index(drop = True)
    while i2 < len(open_close.index):
        prev_cash = cash
        output = oneDay2(open_close, i1, i2, cash)
        cash += output[0]
        temp = pd.DataFrame([[output[1],output[2]]], columns = ['Date','Returns'])
        returns = returns.append(temp, ignore_index=True)
        cash -= prev_cash
        i1 += 2
        i2 += 2
    returns.to_excel('daily_returns.xlsx')
    print("Starting Cash: {0:.2f}".format(float(starting_cash)))
    print("Ending Cash: {0:.2f}".format(float(cash)))
    calculateStats(returns)

In [47]:
#uso_df = uso_df[uso_df['Weekday'] != 2]
uso_df = uso_df.reset_index(drop=True)
uso_df

,Unnamed: 0,index,Time Interval,Close,Net Chg,Open,High,Low,Tick Count,Volume,Date,Weekday
0,0,2,09:30 - 10:00,315.84,10.96,313.92,316.56,313.92,4153.0,214532.0,2012-01-03,1
1,1,3,10:00 - 10:30,316.24,0.40,315.84,316.88,315.68,3300.0,142334.0,2012-01-03,1
2,2,4,10:30 - 11:00,315.76,-0.48,316.32,317.04,315.52,3308.0,125102.0,2012-01-03,1
3,3,5,11:00 - 11:30,314.48,-1.28,315.76,316.24,314.48,2321.0,78503.0,2012-01-03,1
4,4,6,11:30 - 12:00,315.44,0.96,314.48,315.68,314.16,2486.0,77054.0,2012-01-03,1
...,...,...,...,...,...,...,...,...,...,...,...,...
32872,33088,528,13:30 - 14:00,65.11,-0.06,65.18,65.30,65.01,1994.0,458381.0,2022-02-25,4
32873,33089,529,14:00 - 14:30,65.31,0.20,65.11,65.50,65.08,2457.0,471463.0,2022-02-25,4
32874,33090,530,14:30 - 15:00,65.48,0.17,65.35,65.69,65.29,1336.0,293893.0,2022-02-25,4
32875,33091,531,15:00 - 15:30,65.73,0.25,65.48,65.76,65.47,1240.0,238147.0,2022-02-25,4


Set up input for 10amClose/4pm previous Day close signal backtest

In [63]:
# Set up intputs for backtester
start_interval = "09:30 - 10:00"
end_interval = "15:30 - 16:00"
cash = 1000
i1 = 1
i2 = 4
#
moreDays2(uso_df, dates, i1, i2, start_interval, end_interval, cash)

Starting Cash: 1000.00
Ending Cash: 1755.41
Backtest Statistics
Average Arithmetic Daily Returns: 0.023%
Average Geometric Daily Returns: 0.022%
Average Arithmetic Yearly Returns: 5.772%
Standard Deviation of Daily Returns: 0.003
Standard Deviation of Yearly Returns: 0.054
Sharpe ratio: 0.700
